In [2]:
import pandas as pd
from pandas import DataFrame
print('pandas installed.')
!pip install geopy
print('GeoPy installed.')
from geopy.geocoders import Nominatim
print('Nominatim imported.')
from geopy.distance import geodesic
print('Geodesic imported.')
!pip install folium
import folium
print('folium imported.')
import requests
print('requests imported.')
from pandas import json_normalize
print('json_normalize imported.')
import numpy as np
print('numpy imported.')
from sklearn.cluster import KMeans
print('k-means imported.')

pandas installed.
GeoPy installed.
Nominatim imported.
Geodesic imported.
folium imported.
requests imported.
json_normalize imported.
numpy imported.
k-means imported.


In [3]:
# Get Seattle latitude and longitude
address = 'Seattle, WA'
geolocator = Nominatim(user_agent="sea_explorer")
location = geolocator.geocode(address)
latitude_1 = location.latitude
longitude_1 = location.longitude
print('The geographical coordinates of Seattle are {}, {}.'.format(latitude_1, longitude_1))

The geographical coordinates of Seattle are 47.6038321, -122.3300624.


In [4]:
# Get Space Needle latitude and longitude
address = '400 Broad Street, Seattle, WA'
geolocator = Nominatim(user_agent="sea_explorer")
location = geolocator.geocode(address)
latitude_2 = location.latitude
longitude_2 = location.longitude
Space_Needle = (latitude_2, longitude_2)
print('The geographical coordinates of the Space Needle are {}, {}.'.format(latitude_2, longitude_2))

The geographical coordinates of the Space Needle are 47.6205131, -122.34930359883187.


In [5]:
# Create Seattle map
Seattle_map = folium.Map(location=[latitude_1, longitude_1], zoom_start=11)

# Plot the Space Needle
folium.Marker([latitude_2,longitude_2]).add_to(Seattle_map)
Seattle_map

In [6]:
# Foursquare credentials
CLIENT_ID =     'TPEHLLBFIFW4JNBBETGDQPH4KLUKC0M42HT3BP2OO51XYGKZ'
CLIENT_SECRET = 'JXH5RRIYNOBBEZB40PJ5OBXW4I05E2FWTLANB40LOWT3FWJ4'
VERSION = '20200406'
LIMIT = 50
print('CLIENT_ID:  ' + CLIENT_ID)
print('CLIENT_SECRET:  ' + CLIENT_SECRET)

CLIENT_ID:  TPEHLLBFIFW4JNBBETGDQPH4KLUKC0M42HT3BP2OO51XYGKZ
CLIENT_SECRET:  JXH5RRIYNOBBEZB40PJ5OBXW4I05E2FWTLANB40LOWT3FWJ4


In [8]:
# Search for Seattle area hospitals
search_query = 'Hospital'
radius = 32286.9 # 20 miles
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    latitude_1,
    longitude_2,
    VERSION,
    search_query,
    radius,
    LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=TPEHLLBFIFW4JNBBETGDQPH4KLUKC0M42HT3BP2OO51XYGKZ&client_secret=JXH5RRIYNOBBEZB40PJ5OBXW4I05E2FWTLANB40LOWT3FWJ4&ll=47.6038321,-122.34930359883187&v=20200406&query=Hospital&radius=32286.9&limit=50'

In [9]:
# Send the get request
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e8e1e15a2e538001b8fc14f'},
 'response': {'venues': [{'id': '43680180f964a52078291fe3',
    'name': 'Virginia Mason Hospital and Seattle Medical Center',
    'location': {'address': '1100 9th Ave',
     'lat': 47.610128497530766,
     'lng': -122.32723219993456,
     'labeledLatLngs': [{'label': 'display',
       'lat': 47.610128497530766,
       'lng': -122.32723219993456}],
     'distance': 1798,
     'postalCode': '98101',
     'cc': 'US',
     'neighborhood': 'First Hill',
     'city': 'Seattle',
     'state': 'WA',
     'country': 'United States',
     'formattedAddress': ['1100 9th Ave',
      'Seattle, WA 98101',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d196941735',
      'name': 'Hospital',
      'pluralName': 'Hospitals',
      'shortName': 'Hospital',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/medical_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-158637231

In [23]:
# Assign relevant part of JSON to venues
venues = results['response']['venues']

# Transform venues into a dataframe
df1 = json_normalize(venues)
print(len(df1))

50


In [26]:
# Keep selected columns
filtered_columns = ['name', 'categories', 'location.address', 'location.city', 'location.lat', 'location.lng']
df_filtered = df1.loc[:, filtered_columns]
df_filtered.head()

,name,categories,location.address,location.city,location.lat,location.lng
0,Virginia Mason Hospital and Seattle Medical Ce...,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",1100 9th Ave,Seattle,47.610128,-122.327232
1,Kindred Hospital Seattle - First Hill,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",1334 Terry Ave,Seattle,47.611843,-122.328792
2,Overlake Hospital Medical Center,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",1035 116th Ave NE,Bellevue,47.620627,-122.186728
3,Seattle Children's Hospital,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",4800 Sand Point Way NE,Seattle,47.662590,-122.282741
4,Elliott Bay Animal Hospital,"[{'id': '4d954af4a243a5684765b473', 'name': 'V...",2042 15th Ave W,Seattle,47.638149,-122.375836


In [27]:
# Function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
# Filter the category for each row
df_filtered['categories'] = df_filtered.apply(get_category_type, axis=1)
df_filtered

,name,categories,location.address,location.city,location.lat,location.lng
0,Virginia Mason Hospital and Seattle Medical Ce...,Hospital,1100 9th Ave,Seattle,47.610128,-122.327232
1,Kindred Hospital Seattle - First Hill,Hospital,1334 Terry Ave,Seattle,47.611843,-122.328792
2,Overlake Hospital Medical Center,Hospital,1035 116th Ave NE,Bellevue,47.620627,-122.186728
3,Seattle Children's Hospital,Hospital,4800 Sand Point Way NE,Seattle,47.662590,-122.282741
4,Elliott Bay Animal Hospital,Veterinarian,2042 15th Ave W,Seattle,47.638149,-122.375836
5,Cascade Behavioral Hospital,Medical Center,12844 Military Rd S,Tukwila,47.487296,-122.296154
6,Kaiser Permanente Hospital,Hospital,11511 NE 10th St,Bellevue,47.618550,-122.186462
7,Seattle Grace Mercy West Hospital,Hospital,NaN,Seattle,47.609242,-122.335862
8,Northwest Veterinary Hospital,Veterinarian,4922 Stone Way N,Seattle,47.664807,-122.340218
9,Seattle Children’s Hospital Offices,Office,6901 Sand Point Way NE,Seattle,47.678749,-122.263923


In [22]:
# Frequency distribution of 'categories' column
f = df_filtered['categories'].value_counts()
f

Veterinarian            20
Hospital                11
Hospital Ward            4
Medical Center           3
Doctor's Office          3
Maternity Clinic         2
Office                   1
Emergency Room           1
Non-Profit               1
Laundry Service          1
Tunnel                   1
Assisted Living          1
Mental Health Office     1
Name: categories, dtype: int64

In [33]:
# Delete any facilities other than hospitals; e.g, exclude veterinarian facilities and clinics
df_2 = df_filtered.loc[df_filtered['categories'] == 'Hospital']

In [34]:
# Drop NaN rows
df_2 = df_2.dropna()

# Delete duplicate hospitals based on address
df_x = df_2.drop_duplicates(subset=['location.address'])

# Delete suspected duplicate hospital based on 'name'
df_x = df_x.loc[df_x['name']!= "Overlake Hospital Medical Center"]

In [35]:
# Create anonomyous function that will calculate the distance in miles from the Space Needle to each hospital
def dist(x, l1, l2):
    return geodesic(x, (l1, l2)).mi

# Loop through df_x and calculate the distance from the Space Needle to each hospital
df_x['distance'] = df_x[['location.lat', 'location.lng']].apply(lambda x: dist(Space_Needle, *x,), axis=1)

In [36]:
# Sort by ascending distance
df_x.sort_values('distance', inplace=True)

# Reindex
df_x.reset_index(drop=True, inplace=True)
df_x

,name,categories,location.address,location.city,location.lat,location.lng,distance
0,Kindred Hospital Seattle - First Hill,Hospital,1334 Terry Ave,Seattle,47.611843,-122.328792,1.129980
1,Virginia Mason Hospital and Seattle Medical Ce...,Hospital,1100 9th Ave,Seattle,47.610128,-122.327232,1.256082
2,Swedish Medical Center - First Hill Campus,Hospital,747 Broadway,Seattle,47.608403,-122.321890,1.529688
3,Kaiser Permanente Hospital,Hospital,201 16th Ave E,Seattle,47.620243,-122.311679,1.757514
4,Seattle Children's Hospital,Hospital,4800 Sand Point Way NE,Seattle,47.662590,-122.282741,4.255469
5,Kaiser Permanente Hospital,Hospital,11511 NE 10th St,Bellevue,47.618550,-122.186462,7.607533


In [37]:
# Index into dx_x to change spelling
df_x.loc[3, 'name'] = 'Kaiser Permanente Hospital #1'
df_x.loc[5, 'name'] = 'Kaiser Permanente Hospital #2'
df_x

,name,categories,location.address,location.city,location.lat,location.lng,distance
0,Kindred Hospital Seattle - First Hill,Hospital,1334 Terry Ave,Seattle,47.611843,-122.328792,1.129980
1,Virginia Mason Hospital and Seattle Medical Ce...,Hospital,1100 9th Ave,Seattle,47.610128,-122.327232,1.256082
2,Swedish Medical Center - First Hill Campus,Hospital,747 Broadway,Seattle,47.608403,-122.321890,1.529688
3,Kaiser Permanente Hospital #1,Hospital,201 16th Ave E,Seattle,47.620243,-122.311679,1.757514
4,Seattle Children's Hospital,Hospital,4800 Sand Point Way NE,Seattle,47.662590,-122.282741,4.255469
5,Kaiser Permanente Hospital #2,Hospital,11511 NE 10th St,Bellevue,47.618550,-122.186462,7.607533


In [43]:
# Merge hospital bed data
beds = DataFrame({'name': ['Kindred Hospital Seattle - First Hill',
                           'Virginia Mason Hospital and Seattle Medical Center', 
                           'Swedish Medical Center - First Hill Campus', 
                           'Kaiser Permanente Hospital #1',
                           "Seattle Children's Hospital",
                           'Kaiser Permanente Hospital #2'],
                           'beds': [50, 336, 697, 18, 407, 347]})

df_3 = pd.merge(df_x, beds)
df_3

,name,categories,location.address,location.city,location.lat,location.lng,distance,beds
0,Kindred Hospital Seattle - First Hill,Hospital,1334 Terry Ave,Seattle,47.611843,-122.328792,1.129980,50
1,Virginia Mason Hospital and Seattle Medical Ce...,Hospital,1100 9th Ave,Seattle,47.610128,-122.327232,1.256082,336
2,Swedish Medical Center - First Hill Campus,Hospital,747 Broadway,Seattle,47.608403,-122.321890,1.529688,697
3,Kaiser Permanente Hospital #1,Hospital,201 16th Ave E,Seattle,47.620243,-122.311679,1.757514,18
4,Seattle Children's Hospital,Hospital,4800 Sand Point Way NE,Seattle,47.662590,-122.282741,4.255469,407
5,Kaiser Permanente Hospital #2,Hospital,11511 NE 10th St,Bellevue,47.618550,-122.186462,7.607533,347


In [39]:
# Add hospital markers to the Seattle map
for lat, lng in zip(df_3['location.lat'], df_3['location.lng']):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Seattle_map)

# Plot the map
Seattle_map

In [44]:
# Reshape the bed data for k-means
X = beds.values[:,1:]
X = np.nan_to_num(X)
print(X)

# k-means analysis
k_means = KMeans(init="k-means++", n_clusters=3, n_init=3)
k_means.fit(X)
labels = k_means.labels_
print(labels)

[[50]
 [336]
 [697]
 [18]
 [407]
 [347]]
[1 0 2 1 0 0]


In [45]:
# k-means analysis
k_means = KMeans(init="k-means++", n_clusters=3, n_init=4)
k_means.fit(X)
labels = k_means.labels_
print(labels)

[1 0 2 1 0 0]


In [46]:
# k-means analysis
k_means = KMeans(init="k-means++", n_clusters=3, n_init=5)
k_means.fit(X)
labels = k_means.labels_
print(labels)

[2 1 0 2 1 1]
